# 데이터 불러오기

In [ ]:
import pandas as pd

file_path = "C:\Potenup\Drug-Detection-Chatbot\data\\antibiotics_c_code.csv"
data = pd.read_csv(file_path)

data

,제품명,C-Code
0,유한짓정100mgYuhanzid Tab. 100mg,K-000040
1,후라시닐정Flasinyl Tab.,K-000066
2,마이암부톨제피정400mgMyambutol Tab. 400mg,K-000080
3,앰씰린캡슐250mgAmcillin Cap. 250mg,K-000105
4,셉트린정Septrin Tab.,K-000114
...,...,...
338,퍼킨씨알정50-200mgPerkin CR Tab. 50-200mg,K-049428
339,조인트맥스프리미엄정Joint Max Premium Tab.,K-049633
340,콘조탑골드600정Chonjotop Gold 600 Tab.,K-052476
341,콘빅600정Conbic 600 Tab.,K-052542


In [129]:
import re

text = "퍼킨씨알정50-200mgPerkin CR Tab. 50-200mg"

# 패턴: [한글] [숫자옵션] [영문옵션] [숫자옵션]
pattern = re.compile(
    r"^([가-힣]+[가-힣A-Za-z]*)"   # 한글 부분
    r"([0-9\-\.]+mg)?"               # 첫 번째 숫자/용량 (있을 수도 있음)
    r"([가-힣A-Za-z\s\.]+)?"         # 영어 부분 (있을 수도 있음)
    r"([0-9\-\.]+mg)?$"              # 마지막 숫자/용량 (있을 수도 있음)
    r"\s*([A-Za-z]+)?$"
)

m = pattern.match(text)
if m:
    parts = [p.strip() for p in m.groups() if p]  # None 제거 + 공백 정리
    print(parts)
else:
    print("매치 실패")

['퍼킨씨알정', '50-200mg', 'Perkin CR Tab.', '50-200mg']


In [11]:
# 한글 숫자용량 (한글) 영어 용량 (한글)
import re
def split_by_hangul(text):
    return re.split(r"(?<=[가-힣])(?=[A-Za-z0-9])", text)

split_by_hangul(text)

['탐부톨정', '400mg(수출용)Tambutol Tab. 400mg(for export)']

In [ ]:
import re

texts = [
    "400mg(수출용)Tambutol Tab",
    "400정Tambutol Tab",
    "200mgAmoxicillin"
]

pattern = re.compile(
    r"^(\d+(?:mg|정))"
    r"^(\d+(?:mg|정))"       # 1. 용량 (숫자 + mg 또는 정)
    r"(?:\(([^)]+)\))?"      # 2. 괄호 안 내용 (선택적)
    r"(.+)$"                 # 3. 제품명 (나머지)
)

for text in texts:
    m = pattern.match(text)
    if m:
        capacity = m.group(1)          # 용량
        note = f"({m.group(2)})" if m.group(2) else ""   # (수출용) 같은 비고
        product = m.group(3).strip()   # 제품명
        print(capacity, "/", note if note else "(없음)", "/", product)

400mg / (수출용) / Tambutol Tab
400정 / (없음) / Tambutol Tab
200mg / (없음) / Amoxicillin


In [70]:
import re

text = "탐부톨정 400mg(수출용) Tambutol Tab. (for export)"

def parser(text):
    pattern1 = re.compile(
        r"([가-힣]+)\s*"          # 1. 한글명
        r"(\d+(?:\.\-\d+)?(?:mg|정))"           # 2. 용량
        r"(?:\(([^)]+)\))?\s*"      # 3. 한글 비고 (예: 수출용)
        r"([A-Za-z\s\-\.]+)"          # 4. 영문명
        r"(?:\(([^)]+)\)|([A-Za-z가-힣]+))?"
    )

    m = pattern1.match(text)
    if m:
        name_kr = m.group(1)
        capacity = m.group(2)
        note_kr = f"({m.group(3)})" if m.group(3) else ""
        name_en = m.group(4).strip()
        note_en = f"({m.group(5)})" if m.group(5) else ""

        result = [name_kr, capacity, note_kr, name_en, note_en]
        # result = f"{name_kr} / {capacity} / {note_kr} / {name_en} / {capacity} / {note_en}"
        print(f"{text}\n\t{result}")
        return result
    else:
        print(f"{text}")
        return text

In [ ]:
data["제품명"]

In [93]:
errors = [
    (31, '소아용루리드정50mgRulid Pediatric Tab 50mg', '소아용루리드정50mgRulid Pediatric Tab. 50mg'),
    (213, '미니보라30Minivlar 30', '미니보라30Minivlar. 30'),
    (241, '비오플록점안액Vioflox Eye Drops', '비오플록점안액Vioflox Eye Drops.')
]

In [98]:
for err in errors:
    data["제품명"][err[0]] = err[-1]

In [132]:
cnt = 0
error_list = []
for i, x in enumerate(data["제품명"].tolist()):
    name = None 
    dose1 = None 
    e_name = None
    dose2 = None 
    etc = None 

    try:
        idx = x.index(".") + 1
        left, right = x[:idx], x[idx:]
        print(x)

        idx = None
        if "mg" in left:
            idx = x.find("mg") + 2
            lleft, e_name = left[:idx], left[idx:]
            print(f"\t{lleft} / {e_name}")
        
        elif "정" in left:
            idx = x.find("정") + 1
            lleft, e_name = left[:idx], left[idx:]
            print(f"\t{lleft} / {e_name}")

        idx = None
        if "mg" in right:
            idx = x.find("mg") + 2
            dose2, etc = right[:idx], right[idx:]
            print(f"\t{dose2} / {etc}")
        
        elif "정" in right:
            idx = x.find("정") + 1
            dose2, etc = right[:idx], right[idx:]
            print(f"\t{dose2} / {etc}")

        print(" / ".join([name, dose1, e_name, dose2, etc]))
    except:
        cnt += 1
        print("오류오류",x)
        error_list.append((i, x))
        pass

유한짓정100mgYuhanzid Tab. 100mg
	유한짓정100mg / Yuhanzid Tab.
	 100mg / 
오류오류 유한짓정100mgYuhanzid Tab. 100mg
후라시닐정Flasinyl Tab.
	후라시닐정 / Flasinyl Tab.
오류오류 후라시닐정Flasinyl Tab.
마이암부톨제피정400mgMyambutol Tab. 400mg
	마이암부톨제피정400mg / Myambutol Tab.
	 400mg / 
오류오류 마이암부톨제피정400mgMyambutol Tab. 400mg
앰씰린캡슐250mgAmcillin Cap. 250mg
	앰씰린캡슐250mg / Amcillin Cap.
	 250mg / 
오류오류 앰씰린캡슐250mgAmcillin Cap. 250mg
셉트린정Septrin Tab.
	셉트린정 / Septrin Tab.
오류오류 셉트린정Septrin Tab.
탐부톨정400mg(수출용)Tambutol Tab. 400mg(for export)
	탐부톨정400mg / (수출용)Tambutol Tab.
	 400mg(fo / r export)
오류오류 탐부톨정400mg(수출용)Tambutol Tab. 400mg(for export)
씬지로이드정0.1mgSynthyroid Tab. 0.1mg
	씬지로이드정 / 0.
	1mgSynthyro / id Tab. 0.1mg
오류오류 씬지로이드정0.1mgSynthyroid Tab. 0.1mg
에이씰린캡슐250mgA-Cillin Cap. 250mg
	에이씰린캡슐250mg / A-Cillin Cap.
	 250mg / 
오류오류 에이씰린캡슐250mgA-Cillin Cap. 250mg
후시딘정Fucidin Tab. 250mg
	후시딘정 / Fucidin Tab.
	 250mg / 
오류오류 후시딘정Fucidin Tab. 250mg
유한피라진아미드정250mgPyrazinamide Tab. 250mg Yuhan
	유한피라진아미드정250mg / Pyrazinamide Tab.
	 250mg Yuhan / 
오

In [114]:
cnt

0

# 벡타 락탐계 항생제 찾기!

In [2]:
import pandas as pd

data = pd.read_csv("C:\Potenup\Drug-Detection-Chatbot\data\\total_result_drug.csv")
data.head()

,C-Code,구분,제품명,복지부분류코드,volume,name,vol_value,vol_unit,성분/함량,첨가물,제형,성상,KPIC/ATC,구분(전문/일반),효능,용법,주의사항,저장방법,name_ko,name_en
0,K-000059,일반의약품,게루삼정 200T,234.0,200T,게루삼정,200.0,T,"['Calcium Carbonate P.P.T. 침강탄산칼슘 100mg', 'Dri...","['스테아르산마그네슘', '카르복시메틸셀룰로오스칼슘']",정제,흰색의 원형정제 백색의 원형 정제,"['소화기계질환', '소화성궤양 치료제', '제산제', '흡수성', '비흡수성', ...",일반,"위산과다, 속쓰림, 위부불쾌감, 위부팽만감, 체함, 구역, 구토, 위통, 신트림.","만 15세 이상 및 성인: 1회 2정, 1일 3회 만 15세 이상 및 성인: 1회 ...",1. 다음과 같은 사람은 이 약을 복용하지 말 것. 1) 투석요법을 받고 있는 환자...,"기밀용기, 실온(1~30℃)보관",게루삼정,Gelusam Tab
1,K-000069,일반의약품,지엘타이밍정 12T,115.0,12T,지엘타이밍정,12.0,T,['Caffeine Anhydrous 카페인무수물 50mg'],"['경질무수규산', '리보플라빈', '스테아르산마그네슘', '옥수수전분', '유당수...",정제,노란색의 팔각형 정제,"['신경계 질환', '기타 신경계용 약물', '기타', 'N06BC01']",일반,졸음,성인 : 무수카페인으로서 1회 100-300㎎을 1일 1-3회 경구 투여한다. 성인...,1. 다음과 같은 사람은 이 약을 복용하지 말 것 이 약은 유당(젖당)을 함유하고 ...,"밀폐용기, 실온(1~30℃)보관",지엘타이밍정,GL Timing Tab
2,K-000080,전문의약품,마이암부톨제피정400mg,622.0,400mg,마이암부톨제피정,400.0,mg,['Ethambutol Hydrochloride 에탐부톨염산염 400mg'],"['D-소르비톨액', '모노라우린산소르비탄', '백당', '스테아르산', '스테아르...",정제,황색의 원형필름코팅정제 황색원형의 제피정,"['감염성질환', '항결핵제', '항균제', '2차약물', 'J04AK02']",전문,1. 유효균종 결핵균 2. 적응증 폐결핵 및 기타 결핵증,1. 초기치료 염산에탐부톨로서 1일 체중 kg당 15 mg을 24시간마다 1회 경구...,1. 다음 환자에는 투여하지 말 것. 1) 이 약 또는 이 약의 구성성분에 과민반응...,"기밀용기, 실온(1 ~ 30℃)보관",마이암부톨제피정,mg Myambutol Tab
3,K-000112,일반의약품,아로나민골드,316.0,NaN,아로나민골드,NaN,NaN,['Ascorbic Acid D.C. 직타용아스코르브산 72.2mg (아스코르브산(...,"['미결정셀룰로오스', '스테아르산마그네슘', '오파글로스클리어(97W19196)'...",정제,1. 내수용 : 이 약은 적색의 필름코팅정이다. 2. 수출용 : 이 약은 빨간색의 ...,"['비타민 및 영양제류', '수용성 비타민', '비타민C 제제', '비타민B 제제'...",일반,"○ 다음 경우의 비타민 B1, B2, B6, E, C의 보급 ○ 다음 경우의 비타민...",성인 1회 1정 1일 2회 복용한다,1) 다음과 같은 사람은 이 약을 복용하지 말 것. (1) 이 약 및 이 약에 포함...,"기밀용기, [내수용] 실온(1~30℃)보관, [수출용] 25℃ 이하 보관",아로나민골드정,Aronamin Gold Tab
4,K-000114,전문의약품,셉트린정 480MG(삼일),621.0,480MG,셉트린정,480.0,mg,"['Sulfamethoxazole 설파메톡사졸 400mg', 'Trimethopri...","['도큐세이트나트륨', '스테아르산마그네슘', '전분글리콜산나트륨', '포비돈']",정제,흰색 또는 거의 흰색의 원형 정제 백색의 원형 정제,"['감염성질환', '항균제', 'Sulfa제', 'Sulfonamides', '기타...",전문,"○ 유효균종 ○ 유효균종 대장균, 시트로박터, 클레브시엘라, 엔테로박터, 프로테우스...",1. 성인 및 12세 이상의 소아 1. 성인 및 12세 이상의 소아 1회 2정을 1...,1. 경고 1) 이 약을 포함한 설파메톡사졸 및 트리메토프림 투여에 의해 중증 피부...,"차광밀폐용기, 실온보관(1~30℃)",셉트린정,Septrin Tab


In [3]:
data.columns

Index(['C-Code', '구분', '제품명', '복지부분류코드', 'volume', 'name', 'vol_value',
       'vol_unit', '성분/함량', '첨가물', '제형', '성상', 'KPIC/ATC', '구분(전문/일반)', '효능',
       '용법', '주의사항', '저장방법', 'name_ko', 'name_en'],
      dtype='object')

In [4]:
data["KPIC/ATC"][0]

"['소화기계질환', '소화성궤양 치료제', '제산제', '흡수성', '비흡수성', 'A02AH']"

In [5]:
import ast

cnt = 0
new_column1 = []
new_column2 = []
temp1 = False 
temp2 = False
for x in data["KPIC/ATC"]:
    try:
        x_list = ast.literal_eval(x)
        temp1 = x_list[-1]
        if "J01C" in temp1:
            # print(temp)
            cnt += 1
            temp2 = True
        else:
            temp2 = False

        new_column1.append(temp1)
        new_column2.append(temp2)
    except:
        temp1 = None
        temp2 = False

        new_column1.append(temp1)
        new_column2.append(temp2)


    

data["new_ATC"] = new_column1
data["isJ"] = new_column2
print(cnt)

37


In [6]:
data.loc[data["isJ"]].shape

(37, 22)

In [11]:
data.loc[data["isJ"]][['제품명', 'C-Code']].sort_values(by="C-Code")

,제품명,C-Code
1007,앰씰린캡슐250mg,K-000105
1023,아목사펜캅셀 250MG (종근당),K-000271
1025,에이씰린캡슐 500mg,K-000304
1027,앰씰린캅셀500mg,K-000341
1043,일동아목시실린수화물캡슐 250mg,K-000608
1046,곰실린캡슐500mg,K-000653
1059,아목시실린캅셀 500MG (종근당),K-000937
1083,오구멘틴정 375MG (일성),K-001516
1091,파목신캡슐 500mg,K-001670
1158,일동아목시실린수화물캡슐500mg,K-002896


In [212]:
data["new_ATC"].value_counts().sort_index()

new_ATC
A01A          10
A01AB22        3
A01AD         37
A02AA02        5
A02AA04        5
              ..
기타             9
말초혈액순환 개선      5
아연(Zn)         1
인슐린 분비 촉진제     2
흡수성            2
Name: count, Length: 661, dtype: int64

In [136]:
data["성분/함량"][0]

"['Calcium Carbonate P.P.T. 침강탄산칼슘 100mg', 'Dried Aluminum Hydroxide Gel 건조수산화알루미늄겔 200mg', 'Magnesium Carbonate 탄산마그네슘 50mg', 'Sodium Bicarbonate 탄산수소나트륨 50mg']"

In [152]:
text = "['Calcium Carbonate P.P.T. 침강탄산칼슘 100mg', 'Dried Aluminum Hydroxide Gel 건조수산화알루미늄겔 200mg', 'Magnesium Carbonate 탄산마그네슘 50mg', 'Sodium Bicarbonate 탄산수소나트륨 50mg']"
text.lower()

"['calcium carbonate p.p.t. 침강탄산칼슘 100mg', 'dried aluminum hydroxide gel 건조수산화알루미늄겔 200mg', 'magnesium carbonate 탄산마그네슘 50mg', 'sodium bicarbonate 탄산수소나트륨 50mg']"

In [161]:
import json 
import ast

search = "Clarithromycin"

for idx, x in enumerate(data["성분/함량"]):
    try:
        if search.lower() in x.lower():
            print(f"{data['제품명'][idx]} >  {x}")
        # xs = ast.literal_eval(x)
        # print(xs)
    except:
        # print(f"오류오류 {x}")
        pass

클래리정 >  ['Clarithromycin 클래리트로마이신 250mg']
클래리시드엑스엘서방정 500MG (현대) >  ['Clarithromycin 클래리트로마이신 500mg']
클래리정 500mg >  ['Clarithromycin 클래리트로마이신 250mg']
클래리시드필름코팅정 500mg >  ['Clarithromycin 클래리트로마이신 500mg']
클라로마정 250MG (고려) >  ['Clarithromycin 클래리트로마이신 250mg']
클래신정 250mg >  ['Clarithromycin 클래리트로마이신 250mg']
클로신정 250mg >  ['Clarithromycin 클래리트로마이신 250mg']
클라로마정 500MG (고려) >  ['Clarithromycin 클래리트로마이신 250mg']
클래리시드필름코팅정 250mg >  ['Clarithromycin 클래리트로마이신 500mg']
리시브골드캡슐 60 >  ['Clarithromycin 클래리트로마이신 500mg']
클래트론정500mg >  ['Clarithromycin 클래리트로마이신 250mg']


In [186]:
import re

text = "유한피라진아미드정250mg Pyrazinamide Tab. 250mg Yuhan"

def extract_eng_name(text):
    # 영어와 공백만 추출
    result = re.findall(r"[A-Za-z\s]+", text)
    # print(result[0])   # [' ', 'Pyrazinamide Tab', ' ', ' Yuhan']

    return result[0]

In [175]:
# 모든 성분 모아보기 
import ast 

ingredient = []

for x in data["성분/함량"]:
    try:
        x_list = ast.literal_eval(x)
        ingredient.extend(x_list)
    except:
        pass

In [187]:
ingredient_set_list = sorted(set(ingredient))

In [192]:
tests = []
for ing in ingredient_set_list:
    text = extract_eng_name(ing)
    tests.append(text.strip())
    # print(text)

In [194]:
len(set(tests))

969

In [196]:
set(tests)

{'',
 'Acamprosate Calcium',
 'Acanthopanax',
 'Acarbose',
 'Acebrophylline',
 'Aceclofenac',
 'Acetaminophen',
 'Acetaminophen Encapsulated',
 'Acetazolamide',
 'Acetyl',
 'Acetylcysteine',
 'Acrinol Hydrate',
 'Acyclovir',
 'Adefovir dipivoxil',
 'Adenine Hydrochloride',
 'Aesculus Hippocastanum L',
 'Afloqualone',
 'Agnus Castus Fruit',
 'Agomelatine',
 'Akebia Stem',
 'Albendazole',
 'Alfacalcidol',
 'Alfuzosin Hydrochloride',
 'Alginic Acid',
 'Alisma Rhizome',
 'Alitretinoin',
 'Allopurinol',
 'Allylisopropylacetylurea',
 'Almagate',
 'Almotriptan Malate',
 'Aloe',
 'Aloe Ext',
 'Aloe Leaf Dried Ext',
 'Alogliptin Benzoate',
 'Alprazolam',
 'Aluminum Magnesium Hydroxide',
 'Aluminum Magnesium Silicate',
 'Aluminum Silicate Synthetic',
 'Alverine Citrate',
 'Amantadine Sulfate',
 'Ambroxol Hydrochloride',
 'Amiloride Hydrochloride',
 'Aminobenzoate Potassium',
 'Aminophylline',
 'Amiodarone Hydrochloride',
 'Amisulpride',
 'Amitriptyline Hydrochloride',
 'Amlodipine Adipate',
 'Am

In [203]:
search_list = ["테트", "Erythromycin", "Clarithromycin", "Ketoconazole"]
search_list = ["Penicillin", "Amoxicillin", "Ampicillin", "Cloxacillin", "Piperacillin", "Ticarcillin", "Aztreonam","Oxacillin"]
# search_list = ["Imipenem", "Meropenem", "Ertapenem"]
for search in search_list:
    cnt = 0
    print(f"{search} 찾기를 시작합니다.")
    for idx, x in enumerate(data["성분/함량"]):
        try:
            if search.lower() in x.lower():
                print(f"{data['제품명'][idx]} >  {x}")
                cnt += 1
            # xs = ast.literal_eval(x)
            # print(xs)
        except:
            # print(f"오류오류 {x}")
            pass
    if cnt == 0:
        print("찾을 수 없습니다.")
    print("="*100)

Penicillin 찾기를 시작합니다.
찾을 수 없습니다.
Amoxicillin 찾기를 시작합니다.
오구멘틴정 625MG(일성) >  ['Amoxicillin Hydrate 아목시실린수화물 250mg', 'Potassium Clavulanate 클라불란산칼륨 125mg']
크라맥스정375밀리그램_(1정) >  ['Amoxicillin Hydrate 아목시실린수화물 250mg', 'Potassium Clavulanate Diluted 묽은클라불란산칼륨 215.52mg (클라불란산칼륨(으)로서 125mg)']
크라모틴정(아목시실린·클라불란산칼륨(2:1)) >  ['Amoxicillin Hydrate 아목시실린수화물 250mg', 'Potassium Clavulanate Diluted 묽은클라불란산칼륨 215.52mg (클라불란산칼륨(으)로서 125mg)']
썰타목스정 500MG (알보젠) >  ['Amoxicillin Hydrate 아목시실린수화물 250mg', 'Sulbactam Pivoxil 설박탐피복실 250mg']
유크라정375mg >  ['Amoxicillin Hydrate 아목시실린수화물 250mg', 'Potassium Clavulanate Diluted 묽은클라불란산칼륨 178.57mg (클라불란산칼륨(으)로서 125mg)']
아목시클정875/125mg >  ['Amoxicillin Hydrate 아목시실린수화물 875mg', 'Potassium Clavulanate Diluted 묽은클라불란산칼륨 215.52mg (클라불란산칼륨(으)로서 125mg)']
아목클란듀오정 500MG (한미) >  ['Amoxicillin Hydrate 아목시실린수화물 437.5mg', 'Potassium Clavulanate Diluted 묽은클라불란산칼륨 106.36mg (클라불란산칼륨(으)로서 62.5mg)']
맥시부펜ER정 300MG (한미) >  ['Amoxicillin Hydrate 아목시실린수화물 250mg', 'Potassium Clavulanate Dil

In [170]:
data["주의사항"][0]

'1. 다음과 같은 사람은 이 약을 복용하지 말 것. 1) 투석요법을 받고 있는 환자 2) 만 7세 이하의 어린이 2. 이 약을 복용하는 동안 다음의 약을 복용하지 말 것. 1) 테트라사이클린계 항생제 3. 다음과 같은 사람은 이 약 복용하기 전에 의사, 치과의사, 약사와 상의할 것. 1) 신장장애 2) 다른 약물을 복용하고 있는 사람 4. 다음과 같은 사람( 경우) 이 약의 복용을 즉각 중지하고 의사, 치과의사, 약사와 상의할 것. 상담시 이 첨부문서를 소지할 것. 1) 이 약을 복용하는 동안 변비 또는 설사의 증상이 나타날 경우 2) 2주정도 투여하여도 증상의 개선이 없을 경우 5. 기타 이 약의 복용시 주의할 사항 1) 정해진 용법ㆍ용량을 잘 지킬 것. 2) 어린이 투여할 경우에는 보호자의 지도 감독하에 투여할 것. 3) 나트륨 제한 식이를 하는 사람. 6. 저장상의 주의사항 1) 어린이의 손에 닿지 않는 곳에 보관할 것. 2) 직사일광을 피하고 되도록 습기가 적은 서늘한 곳에 보관할 것. 3) 오용을 막고 품질의 보존을 위하여 다른 용기에 바꾸어 넣지 말 것.'

---

# zip 풀기

In [ ]:
import zipfile
import os

def unzip_all_zips(input_dir: str, output_dir: str):
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(input_dir):
        if filename.lower().endswith(".zip"):
            zip_path = os.path.join(input_dir, filename)

            # 압축파일명(확장자 제거)으로 하위 폴더 생성
            extract_folder = os.path.join(output_dir, os.path.splitext(filename)[0])
            os.makedirs(extract_folder, exist_ok=True)

            # 압축 풀기
            with zipfile.ZipFile(zip_path, "r") as zip_ref:
                zip_ref.extractall(extract_folder)

            print(f"압축 해제 완료: {zip_path} → {extract_folder}")

input_dir = "C:/Potenup/Drug-Detection-Chatbot/data/zip"
output_dir = "C:/Potenup/Drug-Detection-Chatbot/data/data_unzip"

unzip_all_zips(input_dir, output_dir)

압축 해제 완료: C:/Potenup/Drug-Detection-Chatbot/data\TS_25_단일.zip → C:/Potenup/Drug-Detection-Chatbot/data/TS_25_단일_unzip\TS_25_단일


In [27]:
import zipfile
import os

def unzip_single_zip(zip_path: str, output_dir: str):
    os.makedirs(output_dir, exist_ok=True)

    # zip 파일명(확장자 제거)으로 하위 폴더 생성
    filename = os.path.basename(zip_path)
    extract_folder = os.path.join(output_dir, os.path.splitext(filename)[0])
    os.makedirs(extract_folder, exist_ok=True)

    # 압축 풀기
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_folder)

    print(f"압축 해제 완료: {zip_path} → {extract_folder}")

zip_path = "C:/Potenup/Drug-Detection-Chatbot/data/zip/TS_21_단일.zip"
output_dir = "C:/Potenup/Drug-Detection-Chatbot/data/data_unzip"

unzip_single_zip(zip_path, output_dir)

압축 해제 완료: C:/Potenup/Drug-Detection-Chatbot/data/zip/TS_21_단일.zip → C:/Potenup/Drug-Detection-Chatbot/data/data_unzip\TS_21_단일


In [28]:
import os
import shutil

def move_selected_folders(input_dir: str, output_dir: str, folder_list: list[str]):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for folder_name in folder_list:
        src_path = os.path.join(input_dir, folder_name)
        dst_path = os.path.join(output_dir, folder_name)

        if os.path.exists(src_path) and os.path.isdir(src_path):
            print(f"Moving: {src_path} -> {dst_path}")
            shutil.move(src_path, dst_path)
        else:
            print(f"Skip (not found): {src_path}")

input_dir = "C:/Potenup/Drug-Detection-Chatbot/data/data_unzip/TS_21_단일/TS_21"
output_dir = "C:/Potenup/Drug-Detection-Chatbot/data/final_unzip"
# 23 폴더
# folders_to_move = ["K-001516", "K-001670"]
# 21 폴더
folders_to_move  = ["K-000105", "K-000271", "K-000304", "K-000341"]

move_selected_folders(input_dir, output_dir, folders_to_move)

Moving: C:/Potenup/Drug-Detection-Chatbot/data/data_unzip/TS_21_단일/TS_21\K-000105 -> C:/Potenup/Drug-Detection-Chatbot/data/final_unzip\K-000105
Moving: C:/Potenup/Drug-Detection-Chatbot/data/data_unzip/TS_21_단일/TS_21\K-000271 -> C:/Potenup/Drug-Detection-Chatbot/data/final_unzip\K-000271
Moving: C:/Potenup/Drug-Detection-Chatbot/data/data_unzip/TS_21_단일/TS_21\K-000304 -> C:/Potenup/Drug-Detection-Chatbot/data/final_unzip\K-000304
Moving: C:/Potenup/Drug-Detection-Chatbot/data/data_unzip/TS_21_단일/TS_21\K-000341 -> C:/Potenup/Drug-Detection-Chatbot/data/final_unzip\K-000341


In [ ]:
# K-000105 TL_21_단일 42GB 서영컴퓨터 > 완료
# K-000271 TL_21_단일
# K-000304 TL_21_단일
# K-000341 TL_21_단일
# K-000608 TL_22_단일 36GB 강사컴퓨터 > 완료
# K-000653 TL_22_단일 
# K-000937 TL_22_단일
# K-001516 TL_23_단일 44GB 서영컴퓨터 > 완료
# K-001670 TL_23_단일
# K-002896 TL_25_단일 33GB 서영컴퓨터 > 완료
# K-002937 TL_25_단일
# K-003244 TL_25_단일
# K-003361 TL_2_단일       X
# K-004072 TL_27_단일 26GB 난혜컴퓨터 > 완료
# K-004469 TL_27_단일
# K-004578 TL_3_단일  93GB X
# K-006677 TL_31_단일 27GB 난혜컴퓨터 > 완료
# K-009429 TL_5_단일  92GB 강사컴퓨터 > 진행중
# K-010281 TL_5_단일
# K-011274 TL_6_단일  94GB 난혜컴퓨터 > 완료
# K-011566 TL_6_단일
# K-011701 TL_36_단일 31GB  난혜컴퓨터 > 완료
# K-011833 TL_7_단일  91GB  X
# K-014206 TL_40_단일 29GB  난혜컴퓨터 > 완료
# K-014383 TL_41_단일 21GB  난혜컴퓨터 > 완료
# K-015197 TL_42_단일 23GB  난혜컴퓨터 > 완료
# K-015270 TL_8_단일  94GB  X
# K-021984 TL_52_단일 25GB  강사컴퓨터 > 완료
# K-022244 TL_52_단일
# K-022362 TL_10_단일 93GB  X
# K-023679 TL_54_단일 21GB  강사컴퓨터 > 진행중
# K-030912 TL_67_단일 23GB  서영컴퓨터 > 완료
# K-035610 TL_75_단일 21GB  난혜컴퓨터 > 완료
# K-035811 TL_76_단일 16GB  난혜컴퓨터 > 완료
# K-042098 VL_5_단일        서버컴퓨터 > 완료
# K-043149 VL_7_단일        서버컴퓨터 > 완료
# K-044927 VL_10_단일       서버컴퓨터 > 완료